In [517]:
from dbconn import querydb, querydbtopandas 
import pandas as pd 

In [521]:
mayor = "2019-10"
menor = "2019-09"

In [519]:
especialidades = querydbtopandas("""select desc_prof_especialidad, id_prof_especialidad from DBA.d_prof_convertir_especialidad""")

In [566]:
df = querydbtopandas(f"""select a11.transac  transac,
a11.fecha  id_tie_dia,
max(a15.desc_tie_dia)  desc_tie_dia,
a11.tran_tipo  tipo_transac,
max(a110.desc_tipo_transac)  desc_tipo_transac,
a11.id_pre_prestador  id_pre_prestador,
max(a18.desc_pre_nombre)  desc_pre_nombre,
max(a18.cuit)  cuit,
max(a18.vigente)  vigente,
a11.id_afiliado  id_afi_afiliado,
max((a13.ape + ',' + a13.nombre))  desc_afi_nombre,
max(LTRIM(RTRIM(a13.contra)))  afi_contrato,
max(a13.antig_fecha)  antig_fecha,
max(a13.inte)  afi_integrante,
max(a13.prepaga)  afi_prepaga,
max(a13.naci_fecha)  naci_fecha,
a11.term_id  term_id,
a11.lectura  lectura,
max(case when a11.lectura ='M'  then 'Manual' when a11.lectura= 'L' then 'Magnética' when a11.lectura= 'C' then 'Centro Atención al Prestador' else 'Sin Asignar'end)  CustCol_19,
a11.recha  recha,
a11.ICD  ICD,
a11.id_pres_prestacion  id_pres_prestacion,
max(a17.desc_pres_prestacion)  desc_pres_prestacion,
max(a17.prestac_pres_prestacion)  prestac_pres_prestacion,
max(a17.nomen_pres_prestacion)  nomen_pres_prestacion,
max(a17.modalidad_cobertura)  modalidad_cobertura,
a11.plan_codi  id_asi_plan_asistencial,
max(a16.desc_asi_plan_asistencial)  desc_asi_plan_asistencial,
a11.descri_copa  descri_copa,
a11.erro_cober  erro_cober,
a11.transac_ref  transac_ref,
a11.baja_fecha  baja_fecha,
a11.transac_rela  transac_rela,
a12.id_pre_prestador  id_pre_prestador0,
max(a14.desc_pre_nombre)  desc_pre_nombre0,
a14.id_pre_especialidad id_pre_especialidad,
sum(a11.canti)  QTRANSAC
from DBA.ft_cm_transac_remotas a11
join DBA.d_pre_demandante_transac a12
 on (a11.id_pre_prestador_demandante = a12.id_pre_prestador_demandante)
join DBA.afi_afiliados a13
 on (a11.id_afiliado = a13.id_afiliado)
join DBA.d_pre_prestador a14
 on (a12.id_pre_prestador = a14.id_pre_prestador)
join DBA.d_tie_dia_consumo a15
 on (a11.fecha = a15.id_tie_dia)
join DBA.d_asi_plan_asistencial a16
 on (a11.plan_codi = a16.id_asi_plan_asistencial)
join DBA.d_pres_prestacion a17
 on (a11.id_pres_prestacion = a17.id_pres_prestacion)
join DBA.d_pre_prestador a18
 on (a11.id_pre_prestador = a18.id_pre_prestador)
join DBA.d_recha_motivos_view a19
 on (a11.recha = a19.id_recha)
join DBA.d_transac_tipo a110
 on (a11.tran_tipo = a110.tipo_transac)
where (cast(a11.fecha as date) >= date("{menor}") and cast(a11.fecha as date) < date("{mayor}")
 and a11.id_pre_prestador in (1262, 1436, 1871, 1875, 24258, 24421, 24436, 24733, 24752, 26615, 27045,
 27053, 40075, 40088, 52270, 66455, 77274, 81416, 91063, 102752, 103431, 110739, 112684, 123153, 174522, 
 199306))
group by a11.transac,
a11.fecha,
a11.tran_tipo,
a11.id_pre_prestador,
a11.id_afiliado,
a11.term_id,
a11.lectura,
a11.recha,
a11.ICD,
a11.id_pres_prestacion,
a11.plan_codi,
a11.descri_copa,
a11.erro_cober,
a11.transac_ref,
a14.id_pre_especialidad,
a11.baja_fecha,
a11.transac_rela,
a12.id_pre_prestador""")

In [567]:
df_ip = df[df["tipo_transac"] == "IP"]
df_ip = df_ip[["id_pre_prestador0","desc_pre_nombre0", "baja_fecha", "recha", "prestac_pres_prestacion", "id_pre_especialidad", "desc_pres_prestacion", "transac_rela"]]

In [568]:
df_id = df[df["tipo_transac"] == "ID"]
df_id = df_id[["transac_ref", "ICD"]]

## Tratamiento de ID

In [569]:
for i in ["0", "E0", "0C", "0CC", "0B", "0CB", "0A", 'e0', '0c', '0cc', '0b', '0cb', '0a']:
    df_id.loc[df_id["ICD"].str.strip() == i, "ICD" ] = 0
    
for i in ["1", "E1","1C", "1CC", "1B", "1CB", 'e1', '1c', '1cc', '1b', '1cb']:
    df_id.loc[df_id["ICD"].str.strip() == i, "ICD" ] = 1
    
for i in ["2", "E2", "2C", "2CC", 'e2', '2c', '2cc']:
    df_id.loc[df_id["ICD"].str.strip() == i, "ICD" ] = 2
    
for i in ["3", "E3", "3C", "3CC", 'e3', '3c', '3cc']:
    df_id.loc[df_id["ICD"].str.strip() == i, "ICD" ] = 3

In [570]:
df_id = df_id[df_id["ICD"].isin([0,1,2,3])]

## Tratamiento IP

In [571]:
#prestadores_validados = pd.read_csv("prestadores_validados.csv")
#prestadores_validados = prestadores_validados.id_pre_prestador.tolist()

In [572]:
#df_ip = df_ip[df_ip["id_pre_prestador0"].isin(prestadores_validados)]

In [573]:
df_ip = df_ip[df_ip["baja_fecha"].isna()]

In [574]:
df_ip = df_ip[df_ip.recha == 0]

In [575]:
#prestaciones detalladas por ellos como imagenes
imagenes = [180101,180102,180103,180104,180107,180109,180110,180111,180112,180113,180114,180116,180117,180118,180121,18010101,
            18010301,18010302,18010303,18010304,18010305,18010306,18010307,18010308,18010309,18010311,18010312,18010313,18010314,
            18010315,18010316,18010318,18010319,18010320,18010321,18010322,18010323,18010401,18010402,
            18010403,18010404,18010405,18010406,18010407,18010408,18010410,18010411,18010418,18010420,18010421,18010601,18010602,18010603,
            18010604,18010607,18010608,18010609,18010610,18010611,18010701,18010901,18011001,18011101,18011201,18011202,18011203,18011204,18011205,18011206,
            18011207,18011208,18011209,18011210,18011211,18011212,18011213,18011214,18011215,
            18011216,18011217,18011218,18011219,18011220,18011221,18011300,18011301,18011302,
            18011303,18011401,18011402,18011403,18011404,18011405,18011501,18011601,18011602,18011603,
            18011604,18011701,18011801,18011901,18011902,18011903,18011904,18011905,18011906,18011907,18011908,
            18011909,18011910,18011911,18011912,18011913,18011914,18011915,18011916,18011917,18011918,18011919,18011920,
            18011921,18011922,18011923,18011924,18011925,18011926,341001,341002,341003,341004,341005,341006,341007,341008,
            341009,341010,341011,341012,341013,34100101,34100102,34100103,34100104,34100105,34100106,34100107,34100108,
            34100109,34100110,34100111,34100401,34100402,34100403,34100404,34100405,34100501,34100601,34100602,34100701,
            34100702,34100801,34100802,34100803,34100901,34100902,34101101,34101201,34101202,34101203,34101204,34101205,
            34101206,34101207,34101208,34101209,34101210,34101211,34101212,34101213,34101214,34101215,34101216,34101217,
            34101218,34101219,34101220,34101221,34101222,34101223,34101224,34101225,34101226,34101227,4101228,
            34101229,34101234,34101235,34101236,34101237, 34101228,34101301,34101302,34101303,34101304,34101305,34101306,34101307,
            34101308,34101309,34101310,34101311,34101312,34101313,34101314,34101315,34101316,34101317,34101318,34101320,34101403,34101404,
            34101405,34101406,34101407,34101408,34101409,34101410,34101411,34101412,34101413,34101414,34101415,34101416,34101417,
            34101418,34101419,34101420,34101421,34101422,34101423,34101424,34101425,34101426,34101427,34101428,34101429,34101430,
            34101432,20012204,20012210,20012404,20012406,20012412]
imagenes = [str(x) for x in imagenes]

In [576]:
df_ip = df_ip[df_ip['prestac_pres_prestacion'].str.strip().isin(imagenes)]

In [577]:
df_ip = pd.merge(df_ip, especialidades, left_on="id_pre_especialidad", right_on="id_prof_especialidad" , how="left")

In [578]:
#elimino la especialidad ginecología  y obstetricia
df_ip = df_ip[~df_ip["desc_prof_especialidad"].isin(["Ginecología"])]

In [579]:
df_ip = df_ip[~df_ip["desc_prof_especialidad"].isin(["Obstetricia"])]

In [580]:
#elimino todas las prestaciones menos la rmn mamaria
df_ip_rmnmamaria = df_ip[df_ip["desc_pres_prestacion"] == "RMN Mamaria"]

In [581]:
df_ip = df_ip[~df_ip["desc_pres_prestacion"].isin([x for x in df_ip["desc_pres_prestacion"] if 'mam' in x.lower()])]

In [582]:
df_ip = pd.concat([df_ip, df_ip_rmnmamaria],axis=0)

In [583]:
df_id = df_id.groupby("transac_ref")["ICD"].unique().reset_index()

In [584]:
def greatest_selection (df_id):
    for i in range(len(df_id.ICD)):
        if len(df_id.ICD[i]) <= 1:
            df_id.ICD[i] = df_id.ICD[i][0]
            
        elif len(df_id.ICD[i]) > 1:
            df_id.ICD[i].sort()
            df_id.ICD[i] = df_id.ICD[i][-1]
    return df_id

In [585]:
df_id = greatest_selection(df_id)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [586]:
final = pd.merge(df_ip, df_id, left_on="transac_rela",right_on= "transac_ref")

In [587]:
final = final[["id_pre_prestador0","desc_pre_nombre0", "desc_prof_especialidad","ICD"]]

In [588]:
final = final[final["id_pre_prestador0"] != 0]

In [589]:
q_transacciones= final.groupby(["id_pre_prestador0"]).count().reset_index()[["id_pre_prestador0","desc_pre_nombre0"]].rename(columns={"desc_pre_nombre0": "Q de transacciones"})

In [590]:
#q_transacciones = q_transacciones[q_transacciones["Q de transacciones"] >= 60]

In [591]:
final_pivot = pd.pivot_table(final, values='id_pre_prestador0', 
               index=['id_pre_prestador0', 'desc_pre_nombre0', "desc_prof_especialidad"],
                columns="ICD", aggfunc="size")

In [592]:
final_pivot.fillna(0,inplace=True)
final_pivot.reset_index(inplace=True)
final_pivot = pd.merge(q_transacciones, final_pivot)

In [593]:
final_pivot["Normalidad"] = (final_pivot[0] + final_pivot[1] )/ final_pivot["Q de transacciones"]

In [594]:
total_esp = final_pivot.groupby("desc_prof_especialidad")["Q de transacciones"].sum().reset_index().rename(columns={"Q de transacciones": "total_esp"})

In [595]:
final_pivot = pd.merge(final_pivot, total_esp)
final_pivot["Participación"] = final_pivot["Q de transacciones"] / final_pivot["total_esp"]
final_pivot["Participación"] = final_pivot["Participación"].mul(100).astype(float).round(2).astype(str).add('%')
final_pivot.rename(columns={"id_pre_prestador0": "N° Prescriptor", "desc_pre_nombre0": "Prescriptor","Q de transacciones": "Total general"},inplace=True)
imagenes_tabla = final_pivot[["N° Prescriptor", "Prescriptor", 0,1,2,3,  "Total general","Normalidad", "Participación" ]]

In [596]:
final_imagenes = imagenes_tabla.sort_values(by= "Total general", ascending= False)

In [ ]:
final_imagenes